Import Viewing activity associated with my Netflix account as a pandas dataframe

In [1]:
import pandas as pd
all_netflix = pd.read_csv('f:\\Python_projects\\netflix_data\\source_data\\ViewingActivity.csv')

Just isolate data associated with my profile.

In [2]:
my_netflix_raw = all_netflix[all_netflix['Profile Name']=='Priyanka']
#set(my_netflix_raw['Attributes'])
#my_netflix_raw.iloc[:,0]


Removing rows or columns we dont need.
0 - Profile name - Remove
1 - Start Time
2 - Duration
3 - Attributes
4 - Title
5 - Supplemental Video Type
6 - Device Type
7 - Bookmark
8 - Latest Bookmark
9 - Country


In [3]:
my_netflix = my_netflix_raw[my_netflix_raw['Supplemental Video Type'].isna()]
#remove columns 5 to 9
my_netflix = my_netflix.drop(my_netflix.columns[5:10], axis =1)
my_netflix = my_netflix.drop('Attributes', axis=1)
my_netflix.to_csv('my_netflix.csv')

#Remove viewing activity of very small duration (< 10 min)
my_netflix = my_netflix[~(my_netflix['Duration'] < '00:10:00')]

#print(my_netflix.head())
#print(my_netflix.sort_values(by='Duration', ascending=True).tail())

Created new dataframes for movies and TV shows

In [4]:
# Define a regular expression pattern to match strings with two ':'
pattern = r'.*?:.*?:.*'

# Extract rows where 'Title' column does not contain two ':' = movies
my_movies=pd.DataFrame()
my_movies = my_netflix[~my_netflix['Title'].str.contains(pattern, na=False)]

my_movies_name= pd.DataFrame()
my_movies_name = my_movies.groupby('Title').size()

print('movies', my_movies_name.head())
print('no of movies = ', len(my_movies_name))
my_movies_name.to_csv('my_movies.csv')

movies Title
101 Dalmatians          2
13 Going on 30          1
3 Idiots                1
Ali Wong: Baby Cobra    6
Ali Wong: Don Wong      4
dtype: int64
no of movies =  130


In [13]:

# Extract rows where 'Title column contains two ':' = shows
my_shows=pd.DataFrame()
my_shows = my_netflix[my_netflix['Title'].str.contains(pattern, na=False)]

#remove the semi colon after show name
my_shows['Show name'] = my_shows['Title'].str.split(':').str[0].str.strip()

my_shows_name = pd.DataFrame()
my_shows_name = my_shows.groupby('Show name').size()

print('shows' , my_shows_name.head())
print('no of shows = ', len(my_shows_name))

my_shows_name.to_csv('my_shows.csv')


shows Show name
1000 Ways to Die                   2
A Series of Unfortunate Events    50
Afflicted                          3
Aggretsuko                        44
Alias Grace                        8
dtype: int64
no of shows =  136


C:\Users\aandh\AppData\Local\Temp\ipykernel_9892\2913713005.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_shows['Show name'] = my_shows['Title'].str.split(':').str[0].str.strip()


In [12]:
my_movies.head()

,Profile Name,Start Time,Duration,Title
16727,Priyanka,2023-09-01 11:10:01,00:18:42,Flavors of Youth: International Version
16728,Priyanka,2023-08-31 17:52:10,00:27:39,Flavors of Youth: International Version
16729,Priyanka,2023-08-31 13:10:41,00:16:39,Flavors of Youth: International Version
16739,Priyanka,2023-08-29 12:25:40,00:31:47,Easy A
16743,Priyanka,2023-08-28 12:46:26,00:16:05,The Great Hack


In [14]:
my_shows.head()

,Profile Name,Start Time,Duration,Title,Show name
16696,Priyanka,2023-10-30 17:59:30,00:22:26,The Disastrous Life of Saiki K.: Season 1: Epi...,The Disastrous Life of Saiki K.
16697,Priyanka,2023-10-27 10:42:07,00:21:12,The Disastrous Life of Saiki K.: Season 1: Epi...,The Disastrous Life of Saiki K.
16699,Priyanka,2023-10-18 19:21:22,00:19:33,Black Mirror: Season 6: Beyond the Sea (Episod...,Black Mirror
16705,Priyanka,2023-10-05 09:29:20,00:14:03,Derry Girls: Season 3: Episode 2,Derry Girls
16706,Priyanka,2023-10-03 19:10:41,00:25:53,Sex Education: Season 4: Episode 8,Sex Education
